In [1]:
!nvidia-smi

Wed Feb 23 06:23:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import os
import keras.metrics
import matplotlib.pyplot as plt
from imutils import paths
from sklearn.model_selection import train_test_split

In [4]:
test_dir =  '/content/drive/MyDrive/Major Project/Datasets/Phase 1-Fruit Detection/fruits-360/Training/'

In [5]:
fruits = []
fruits_image = []
for i in os.listdir(test_dir):
    for image_filename in os.listdir(test_dir + i):
        fruits.append(i) # name of the fruit 
        fruits_image.append(i + '/' + image_filename)

In [6]:
test_fruits = pd.DataFrame(fruits, columns=["Fruits"])
test_fruits["Fruits Image"] = fruits_image

In [7]:
test_fruits

,Fruits,Fruits Image
0,S1,S1/r_115_100.jpg
1,S1,S1/168_100.jpg
2,S1,S1/158_100.jpg
3,S1,S1/211_100.jpg
4,S1,S1/r_182_100.jpg
...,...,...
66097,S131,S131/17_100.jpg
66098,S131,S131/18_100.jpg
66099,S131,S131/19_100.jpg
66100,S131,S131/20_100.jpg


In [8]:
image_paths=[]
data_labels=[]

In [9]:
for i in range(len(test_fruits)):
  path=test_fruits['Fruits Image'][i]
  path=test_dir+path
  label=test_fruits['Fruits'][i]
  image_paths.append(path)
  data_labels.append(label)

In [10]:
# Training and Validation split
from sklearn.model_selection import train_test_split
(train_data_path_list, image_paths, train_data_class_list, data_labels) = train_test_split(image_paths,
                                                                                                        data_labels,
                                                                                                        test_size = 0.1,
                                                                                                        stratify = data_labels,
                                                                                                        random_state = 101)

In [11]:
df_test = pd.DataFrame({"Path":image_paths,"Class":data_labels})

In [12]:
df_test

,Path,Class
0,/content/drive/MyDrive/Major Project/Datasets/...,S94
1,/content/drive/MyDrive/Major Project/Datasets/...,S115
2,/content/drive/MyDrive/Major Project/Datasets/...,S45
3,/content/drive/MyDrive/Major Project/Datasets/...,S42
4,/content/drive/MyDrive/Major Project/Datasets/...,S102
...,...,...
6606,/content/drive/MyDrive/Major Project/Datasets/...,S13
6607,/content/drive/MyDrive/Major Project/Datasets/...,S47
6608,/content/drive/MyDrive/Major Project/Datasets/...,S25
6609,/content/drive/MyDrive/Major Project/Datasets/...,S39


In [13]:
from keras.models import load_model

trained_model = load_model("/content/drive/MyDrive/Major Project/Training Files/Dataset1/Transfer Learning/Ratio 6/Models/vgg16-ep15.h5")

In [14]:
trained_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [16]:
ts_data = ImageDataGenerator()
testing_df = ts_data.flow_from_dataframe(df_test, x_col = "Path",y_col = "Class", batch_size=1, target_size=(100, 100))

Found 6611 validated image filenames belonging to 131 classes.


In [17]:
trained_model.evaluate(testing_df)

6611/6611 [==============================] - 1683s 253ms/step - loss: 0.0033 - accuracy: 0.9992 - precision: 0.9994 - recall: 0.9992 - auc: 0.9999 - tp: 6606.0000 - fp: 4.0000 - tn: 859426.0000 - fn: 5.0000


[0.0032712058164179325,
 0.9992436766624451,
 0.9993948340415955,
 0.9992436766624451,
 0.9999237656593323,
 6606.0,
 4.0,
 859426.0,
 5.0]

In [18]:
from keras.models import load_model

trained_model = load_model("/content/drive/MyDrive/Major Project/Training Files/Dataset1/Transfer Learning/Ratio 6/Models/vgg16-ep10.h5")

In [19]:
trained_model.evaluate(testing_df)

6611/6611 [==============================] - 148s 22ms/step - loss: 0.0092 - accuracy: 0.9976 - precision: 0.9983 - recall: 0.9967 - auc: 1.0000 - tp: 6589.0000 - fp: 11.0000 - tn: 859419.0000 - fn: 22.0000


[0.009176463820040226,
 0.99757981300354,
 0.9983333349227905,
 0.9966722130775452,
 0.9999998211860657,
 6589.0,
 11.0,
 859419.0,
 22.0]